This notebook test the variance of 
$$I_{\varphi}(\epsilon) = \sum_{x \in \mathcal{X}_{\epsilon}} \varphi(x)$$
where $\varphi : \mathbb{R}^d \mapsto \mathbb{R}$ is a function, $\mathcal{X}_{0}$ is a stationary Poisson point process of intensity $\rho$ and $\mathcal{X}_{\epsilon}$ is tha corresponding pushed point process at time $t$ and with step $\epsilon$.

The goals is to show that $Var(I_{\varphi}(\epsilon)) < Var(I_{\varphi}(0)$ for $0< \epsilon <\epsilon_0 $

    1- Function to be used as test functions
    2- Variance of linear statistics of push for d = 3 and a wide range of epsilons
    3- Variance of linear statistics of push for d = 3 and a wide range of negative and positive epsilons
    4- The bump function
    5- Changing the Force function (useless)
    6- Tests for Equi pp 
    7- Using a truncated version of the force
    8- Test if $\epsilon= 1/(2\rho \kappa_d d)$ is a spetial value of epsilon

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# to delete this cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

# 1- Test functions

In [ ]:
def f_1(x) :
    return np.linalg.norm(x, axis=1)**2

def f_2(x):
    return np.sum(x, axis=1)/(f_1(x) +1)
#indicator function
def f_3(x, window):
    return np.array([int(window.indicator_function(p) == True) for p in x])

def f_4(x):
    return -3*np.cos(f_1(x)) + 4*np.sin(f_2(x))

def bump_indicator(n, points, R=1):
    norm_points = np.linalg.norm(points, axis=1)
    f = np.zeros((points.shape[0]))
    f[norm_points<R]= np.exp(-1/(n*(1- (norm_points[norm_points<R]/R)**2)))
    return f


In [ ]:
import matplotlib.pyplot as plt

x = np.arange(-4, 4, 0.1)
z_f_1 = f_1(np.array([x, x]).T)
z_f_2 = f_2(np.array([x, x]).T)
z_f_4 = f_4(np.array([x, x]).T)
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(x, x, z_f_1, c=z_f_1, cmap='Greens')
ax.set_title("f_1")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(x, x, z_f_2, c=z_f_2, cmap='Greens')
ax.set_title("f_2")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(x, x, z_f_4, c=z_f_4, cmap='Greens')
ax.set_title("f_4")
plt.show()

In [ ]:
x = np.linspace(-2,2, 80)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points)
z_2 = bump_indicator(100, points)
z_3 = bump_indicator(6000, points)
window = BallWindow(center=[0,0], radius=1)
z_4 = f_3(points, window)

fig = plt.figure(figsize=(12, 3))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.scatter3D(X.ravel(), Y.ravel(), z_4, c=z_4)
ax.set_title("n=10")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_2, c=z_2)
ax.set_title("n=100")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
ax.set_title("n=1000")
plt.show()

# 2- Variance of linear statistics of push for d = 3 and a wide range of positive epsilons

In [ ]:
# Creat a poisson point process
d = 3
simu_window = BallWindow(center=[0]*d, radius=8)
window = BallWindow(center=[0]*d, radius=4)
intensity = 6
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-0.07, -0.04, -epsilon_th, -0.004, -0.001, 0.001, 0.004, epsilon_th, 0.006, 0.008, 0.01, 0.04, 0.07]
means_f1_push, stds_f1_push, means_f2_push, stds_f2_push, means_f4_push, stds_f4_push = [], [], [], [], [], []
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")
for epsilon in epsilons:
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
        push_pp = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1) for g in gpp_pp]
        push_pp_res = [p.restrict_to_window(window) for p in push_pp]
        # mean number of points in the restricted window
        pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
        print("nb_pts_poi", pois_res_pt_nb, 
              "nb_pts_pu", push_res_pt_nb)
        f1_push = [np.sum(f_1(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f1_push.append(stat.mean(f1_push))
        stds_f1_push.append(stat.stdev(f1_push))
        f2_push = [np.sum(f_2(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f2_push.append(stat.mean(f2_push))
        stds_f2_push.append(stat.stdev(f2_push))
        f4_push = [np.sum(f_4(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f4_push.append(stat.mean(f4_push))
        stds_f4_push.append(stat.stdev(f4_push))
        print("std poisson f_1", std_f1_poisson, "std push", stds_f1_push)
        print("std poisson f_2", std_f2_poisson, "std push", stds_f2_push)
        print("std poisson f_4", std_f4_poisson, "std push", stds_f4_push)
        print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(x, x, z_f_1, c=z_f_1, cmap='Greens')
ax.set_title("f_1")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(x, x, z_f_2, c=z_f_2, cmap='Greens')
ax.set_title("f_2")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(x, x, z_f_4, c=z_f_4, cmap='Greens')
ax.set_title("f_4")
plt.show()

epsilons.append(0)
stds_f1_push.append(std_f1_poisson)
stds_f2_push.append(std_f2_poisson)
stds_f4_push.append(std_f4_poisson)
epsilons = epsilons[:12]
stds_f1_push = stds_f1_push[:12]
stds_f2_push = stds_f2_push[:12]
stds_f4_push = stds_f4_push[:12]
fig, ax = plt.subplots(1,3, figsize=(15, 5))

ax[0].plot(epsilons, stds_f1_push, "b.", label="f_1" )
ax[0].plot(epsilons, stds_f1_push, "b" )
ax[0].plot(0, std_f1_poisson, "r.", label="f_1 poisson")
ax[0].legend()
ax[0].set_title("std f_1")
ax[1].plot(epsilons, stds_f2_push,  "k.", label="f_2")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(0, std_f2_poisson, "r.", label="f_2 poisson")
ax[1].legend()
ax[1].set_title("std f_2")
ax[2].plot(epsilons, stds_f4_push,  "k.", label="f_4")
ax[2].plot(epsilons, stds_f4_push,  "k")
ax[2].plot(0, std_f4_poisson, "r.", label="f_4 poisson")
ax[2].legend()
ax[2].set_title("std f_4")
plt.show()
plt.show()

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

d = 3
simu_window = BallWindow(center=[0]*d, radius=10)
window = BallWindow(center=[0]*d, radius=4)
intensity = 6
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson_2 = stat.mean(f1_poisson)
std_f1_poisson_2 = stat.stdev(f1_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson_2 = stat.mean(f2_poisson)
std_f2_poisson_2 = stat.stdev(f2_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson_2 = stat.mean(f4_poisson)
std_f4_poisson_2 = stat.stdev(f4_poisson)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-epsilon_th, -0.004, -0.001, 0.001, 0.004, epsilon_th, 0.006, 0.008, 0.01, 0.04, 0.07]
means_f1_push_2, stds_f1_push_2, means_f2_push_2, stds_f2_push_2, means_f4_push_2, stds_f4_push_2 = [], [], [], [], [], []
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")
for epsilon in epsilons:
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
        push_pp = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1) for g in gpp_pp]
        push_pp_res = [p.restrict_to_window(window) for p in push_pp]
        # mean number of points in the restricted window
        pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
        print("nb_pts_poi", pois_res_pt_nb, 
              "nb_pts_pu", push_res_pt_nb)
        f1_push = [np.sum(f_1(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f1_push_2.append(stat.mean(f1_push))
        stds_f1_push_2.append(stat.stdev(f1_push))
        f2_push = [np.sum(f_2(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f2_push_2.append(stat.mean(f2_push))
        stds_f2_push_2.append(stat.stdev(f2_push))
        f4_push = [np.sum(f_4(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f4_push_2.append(stat.mean(f4_push))
        stds_f4_push_2.append(stat.stdev(f4_push))
        print("std poisson f_1", std_f1_poisson_2, "std push", stds_f1_push_2)
        print("std poisson f_2", std_f2_poisson_2, "std push", stds_f2_push_2)
        print("std poisson f_4", std_f4_poisson_2, "std push", stds_f4_push_2)
        print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(-4, 4, 0.1)
z_f_1 = f_1(np.array([x, x]).T)
z_f_2 = f_2(np.array([x, x]).T)
z_f_4 = f_4(np.array([x, x]).T)
fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(x, x, z_f_1, c=z_f_1, cmap='Greens')
ax.set_title("f_1")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(x, x, z_f_2, c=z_f_2, cmap='Greens')
ax.set_title("f_2")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(x, x, z_f_4, c=z_f_4, cmap='Greens')
ax.set_title("f_4")
plt.show()

fig, ax = plt.subplots(1,3, figsize=(20, 5))
ax[0].plot(epsilons, stds_f1_push_2, "b.", label="f_1 from 25132pts" )
ax[0].plot(epsilons, stds_f1_push_2, "b" )
ax[0].hlines(std_f1_poisson_2, xmin = epsilon_th, xmax=max(epsilons), color="b",linestyle="dashed", label="f_1 poisson 25132pts")
ax[0].plot(epsilons, stds_f1_push, "k.", label="f_1 from 12867pts" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].hlines(std_f1_poisson, xmin = epsilon_th, xmax=max(epsilons), color="k",linestyle="dashed", label="f_1 poisson 12867pts")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$std(\sum_{x} f(x)/N)$ ")
ax[0].set_title("std f_1")
ax[1].plot(epsilons, stds_f2_push_2,  "b.", label="f_2 from 25132pts")
ax[1].plot(epsilons, stds_f2_push_2,  "b")
ax[1].hlines(std_f2_poisson_2, xmin = epsilon_th, xmax=max(epsilons), color="b",linestyle="dashed", label="f_2 poisson 25132pts")
ax[1].plot(epsilons, stds_f2_push,  "k.", label="f_2 from 12867pts")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].hlines(std_f2_poisson, xmin = epsilon_th, xmax=max(epsilons), color="k",linestyle="dashed", label="f_2 poisson 12867pts")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[1].set_ylabel(r"$std(\sum_{x} f(x)/N)$ ")
ax[1].set_title("std f_2")
ax[2].plot(epsilons, stds_f4_push_2,  "b.", label="f_4 from 25132pts")
ax[2].plot(epsilons, stds_f4_push_2,  "b")
ax[2].hlines(std_f4_poisson_2, xmin = epsilon_th, xmax=max(epsilons), color="b",linestyle="dashed", label="f_4 poisson 25132pts")
ax[2].plot(epsilons, stds_f4_push,  "k.", label="f_4 from 12867pts")
ax[2].plot(epsilons, stds_f4_push,  "k")
ax[2].hlines(std_f4_poisson, xmin = epsilon_th, xmax=max(epsilons), color="k",linestyle="dashed", label="f_4 poisson 12867pts")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[2].set_ylabel(r"$std(\sum_{x} f(x)/N)$ ")
ax[2].set_title("std f_4")
plt.show()

We are working in $\mathbb{R}^3$.
We compute the variance of the linear statistics of f_1 and f_2 using the father Poisson pp of intensity $\rho=6$ and the child Push for different epsilon.
We simulate the father Poisson in a Ball window of radius $R=6$ in the first experiment and $R=10$ in the second experiment.
We apply the dynamic to push the points then evaluate the linear statistic $\sum_{x} f(x)$ with $x \in B(0, 4)$  for both experiments.

    1- We vary the radius to see the effect of starting with a PPP in $\mathbb{R}^d$ not a restricted window, 
    i.e. to see if what we obtain holds with an infinite number of points.
    2- We remark that for $R=6$ and $R=10$ the std decrease similarly up to $\epsilon \approx 0.01$
    3- We remark that the mean number of points in the restricted window decrease dramatically as $\epsilon$ increase, that's why I wonder what happens when we have an infinite number of points. I think the behavior may change totally i.e., if a point leaves the window another will replace it and the mean number of point will be constant whatever the value of $\epsilon$.
    4- comparing the mean number of points in the restricted window it seems for $R=10$ (starting from a bigger window) the mean number of points of push in the restricted window decrease less than for $R=6$. 



# 3- Variance of linear statistics of push for d = 3 and a wide range of negative and positive epsilons


In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

d = 3
simu_window = BallWindow(center=[0]*d, radius=6)
window = BallWindow(center=[0]*d, radius=4)
intensity = 6
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-epsilon_th, -0.004, -0.001, -0.0005, -0.0001, 0.0001, 0.0005, 0.001, 0.004, 0.005, epsilon_th]
means_f1_push, stds_f1_push, means_f2_push, stds_f2_push, means_f4_push, stds_f4_push = [], [], [], [], [], []
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")
for epsilon in epsilons:
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
        push_pp = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1) for g in gpp_pp]
        push_pp_res = [p.restrict_to_window(window) for p in push_pp]
        # mean number of points in the restricted window
        pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
        print("nb_pts_poi", pois_res_pt_nb, 
              "nb_pts_pu", push_res_pt_nb)
        f1_push = [np.sum(f_1(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f1_push.append(stat.mean(f1_push))
        stds_f1_push.append(stat.stdev(f1_push))
        f2_push = [np.sum(f_2(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f2_push.append(stat.mean(f2_push))
        stds_f2_push.append(stat.stdev(f2_push))
        f4_push = [np.sum(f_4(p.points))/p.points.shape[0] for p in push_pp_res]
        means_f4_push.append(stat.mean(f4_push))
        stds_f4_push.append(stat.stdev(f4_push))
        print("std poisson f_1", std_f1_poisson, "std push", stds_f1_push)
        print("std poisson f_2", std_f2_poisson, "std push", stds_f2_push)
        print("std poisson f_4", std_f4_poisson, "std push", stds_f4_push)
        print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
fig = plt.figure(figsize=(19, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(x, x, z_f_1, c=z_f_1, cmap='Greens')
ax.set_title("f_1")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(x, x, z_f_2, c=z_f_2, cmap='Greens')
ax.set_title("f_2")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(x, x, z_f_4, c=z_f_4, cmap='Greens')
ax.set_title("f_3")
plt.savefig("linear_stat_used_for_var_comp.pdf")
plt.show()

epsilons.append(0)
stds_f1_push.append(std_f1_poisson)
stds_f2_push.append(std_f2_poisson)
stds_f4_push.append(std_f4_poisson)
epsilons = epsilons[:11]
stds_f1_push = stds_f1_push[:11]
stds_f2_push = stds_f2_push[:11]
stds_f4_push = stds_f4_push[:11]

fig, ax = plt.subplots(1,3, figsize=(18, 5))
ax[0].plot(epsilons, stds_f1_push, "k.", label="f_1" )
ax[0].plot(epsilons, stds_f1_push, "k" )
ax[0].plot(0, std_f1_poisson, "r.", label="f_1 poisson")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[0].set_title("f_1")
ax[1].plot(epsilons, stds_f2_push,  "k.", label="f_2")
ax[1].plot(epsilons, stds_f2_push,  "k")
ax[1].plot(0, std_f2_poisson, "r.", label="f_2 poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$std(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("f_2")
ax[2].plot(epsilons, stds_f4_push,  "k.", label="f_3")
ax[2].plot(epsilons, stds_f4_push,  "k")
ax[2].plot(0, std_f4_poisson, "r.", label="f_3 poisson")
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_ylabel(r"$std(\sum_{x} f(x)/N)$ ")
ax[2].legend()
#ax[2].set_title("f_3")
plt.savefig("variance_linear_stat.pdf")
plt.show()

# 4- The bump function

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(-2,2, 80)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points)
z_2 = bump_indicator(100, points)
z_3 = bump_indicator(6000, points)

fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("n=10")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_2, c=z_2)
ax.set_title("n=100")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
ax.set_title("n=1000")
plt.show()

In [ ]:
x = np.arange(-2, 2, 0.01)
points = np.atleast_2d(x).T
fig = plt.figure(figsize=(7, 5))
plt.plot(x, bump_indicator(10, points), 'b', label="n=10")
plt.plot(x, bump_indicator(100, points), 'g', label="n=100")
plt.plot(x, bump_indicator(200, points), 'grey', label="n=200")
plt.plot(x, bump_indicator(8000, points), 'k', label="n=8000")
plt.plot(x, np.ones_like(x), "r--", "y=1")
plt.legend()
plt.show()

In [ ]:
x = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points, R=8)
z_2 = bump_indicator(100, points, R=8)
z_3 = bump_indicator(2000, points, R=8)

fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("n=10")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_2, c=z_2)
ax.set_title("n=100")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
ax.set_title("n=1000")
plt.savefig("bump_function_n.pdf")
plt.show()


In [ ]:
x = np.linspace(-10, 10, 300)
points = np.atleast_2d(x).T
fig = plt.figure(figsize=(7, 5))
plt.plot(x, bump_indicator(10, points, R=8), 'b', label="n=10")
plt.plot(x, bump_indicator(100, points, R=8), 'g', label="n=100")
plt.plot(x, bump_indicator(200, points, R=8), 'grey', label="n=200")
plt.plot(x, bump_indicator(1000, points, R=8), 'k', label="n=1000")
plt.legend()
plt.show()


 trying to understand the derivative 

In [ ]:
def grad_bump_indicator(n, points, R=1):
    d = points.shape[1]
    norm_points = np.linalg.norm(points, axis=1)
    grad_f = np.zeros_like(points)
    norm_square = np.atleast_2d(norm_points[norm_points<R]**2).T
    deno = 1 - norm_square/(R**2)
    f_points =  np.atleast_2d(bump_indicator(n, points[norm_points<R], R)).T
    grad_f[norm_points<R]= -2/(n*deno**2)*f_points*points[norm_points<R]
    return grad_f



In [ ]:
n = 1
x = np.linspace(-6, 6, 400)
points = np.atleast_2d(x).T
f_ = bump_indicator(n, points, R=4)
grad_f = grad_bump_indicator(n, points, R=4)

fig, axis = plt.subplots(1, 2, figsize=(18, 5))
axis[0].plot(x, f_, 'b.')
axis[1].plot(x, grad_f, 'b.')
plt.show()

x = np.linspace(-6, 6, 100)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(n, points, R=4)
z_2 = grad_bump_indicator(n, points, R=4)

fig = plt.figure(figsize=(20, 5))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("f")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_2[:,0], c=z_2[:, 0])
ax.set_title("Component of grad(f)")
ax.view_init(10, 45)
plt.show()

In [ ]:
x = np.linspace(-6, 6, 400)
points = np.atleast_2d(x).T
f_ = 1/np.sqrt(2*np.pi)*np.exp(-points**2/2)
grad_f = -f_*points

fig, axis = plt.subplots(1, 3, figsize=(15, 5))
axis[0].plot(x, f_, 'b.')
axis[1].plot(x, grad_f, 'b.')
axis[2].plot(x, f_, 'b.')
axis[2].plot(x, grad_f, 'k.')
plt.show()

variance of the linear statistic associated with the bump function

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

d = 3
simu_window = BallWindow(center=[0]*d, radius=6)
window = BallWindow(center=[0]*d, radius=4)
intensity = 6
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
n=10
f5_poisson = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in poisson_pp_res]
mean_f5_poisson = stat.mean(f5_poisson)
std_f5_poisson = stat.stdev(f5_poisson)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-epsilon_th, -0.004, -0.001, -0.0005, -0.0001, 0.0001, 0.0005, 0.001, 0.004, 0.005, epsilon_th]
means_f5_push, stds_f5_push = [], []
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")
for epsilon in epsilons:
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
        push_pp = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1) for g in gpp_pp]
        push_pp_res = [p.restrict_to_window(window) for p in push_pp]
        # mean number of points in the restricted window
        pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
        print("nb_pts_poi", pois_res_pt_nb, 
              "nb_pts_pu", push_res_pt_nb)
        f5_push = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in push_pp_res]
        means_f5_push.append(stat.mean(f5_push))
        stds_f5_push.append(stat.stdev(f5_push))
        print("std poisson f_5", std_f4_poisson, "std push", stds_f5_push)
        print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
x = np.linspace(-10, 10, 100)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points, R=3)

fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("Bump function (n=10)")
plt.savefig("bump_function.pdf")
plt.show()
fig, ax = plt.subplots( figsize=(7, 5))
ax.plot(epsilons, stds_f5_push, "k." )
ax.plot(epsilons, stds_f5_push, "k" )
ax.plot(0, std_f5_poisson, "r.", label="For Poisson")
ax.legend()
ax.set_xlabel(r"$\epsilon$")
ax.set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax.set_title("Variance linear statistics with the bump function (n=10)")
plt.savefig("std_ls_bump_fun.pdf")
plt.show()

# 5- Change the Force to reduce the variance directly on 0 and not having a critical point
At the time I thought thet we have a saddle point at 0, hear I tried to see if I can modify the function to get non zero derivative of the variance but this is useless now

In [ ]:
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

d = 2
simu_window = BallWindow(center=[0]*d, radius=6)
intensity = 30
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window) 
#push
gpp_pp = GravityPointProcess(poisson_pp)
epsilon_th = gpp_pp.epsilon
print("epsilon_th", epsilon_th)
epsilons =epsilon_th
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")


if __name__ == "__main__":
    gpp_pp = GravityPointProcess(poisson_pp)
    push_pp_modif = gpp_pp.pushed_point_pattern(epsilon=epsilon_th, stop_time=5, modif=True)
    gpp_pp = GravityPointProcess(poisson_pp)
    push_pp = gpp_pp.pushed_point_pattern(epsilon=epsilon_th, stop_time=5, modif=False)

time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
fig, axis= plt.subplots(1, 3, figsize=(16,8))
poisson_pp.plot(axis=axis[0], c='b', s=1, label="poisson")
axis[0].set_title("Poisson")
push_pp.plot(axis=axis[1], c='r', s=1, label="push")
axis[1].set_title("Push")
push_pp_modif.plot(axis=axis[2], c='k', s=1, label="modif")
axis[2].set_title("Push modif")
plt.show()

fig, axis= plt.subplots(figsize=(8,8))
poisson_pp.plot(axis=axis, c='b', s=1, label="poisson")
push_pp.plot(axis=axis, c='r', s=1, label="push")
push_pp_modif.plot(axis=axis, c='k', s=1, label="modif")
axis.legend()
plt.show()


In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

d = 3
simu_window = BallWindow(center=[0]*d, radius=6)
window = BallWindow(center=[0]*d, radius=4)
intensity = 6
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
n=10
f5_poisson = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in poisson_pp_res]
mean_f5_poisson = stat.mean(f5_poisson)
std_f5_poisson = stat.stdev(f5_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-epsilon_th, -0.004, -0.001, -0.0005, -0.0001, 0.0001, 0.0005, 0.001, 0.004, 0.005, epsilon_th]
means_f5_push, stds_f5_push, means_f5_push_modif, stds_f5_push_modif = [], [], [], []
means_f4_push, stds_f4_push, means_f4_push_modif, stds_f4_push_modif = [], [], [], []
means_f2_push, stds_f2_push, means_f2_push_modif, stds_f2_push_modif = [], [], [], []
means_f1_push, stds_f1_push, means_f1_push_modif, stds_f1_push_modif = [], [], [], []
time_strat = time.time() 
print("start_time", time.time() - time_strat)
print("----------------------------------------------")
for epsilon in epsilons:
    print("epsilon", epsilon)
    if __name__ == "__main__":
        gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
        #push
        push_pp = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1, modif=False) for g in gpp_pp]
        push_pp_res = [p.restrict_to_window(window) for p in push_pp]
        #push modif 
        push_pp_modif = [g.pushed_point_pattern(epsilon=epsilon, stop_time=1, modif=True) for g in gpp_pp]
        push_pp_res_modif = [p.restrict_to_window(window) for p in push_pp_modif]
        # mean number of points in the restricted window
        pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
        push_res_modif_pt_nb = stat.mean(push_pp_res_modif[i].points.shape[0] for i in range(samples_nb))
        
        print("nb_pts_poi", pois_res_pt_nb, 
              "nb_pts_push", push_res_pt_nb,
             "nb_pts_push_modif", push_res_modif_pt_nb,)
        #for f_5
        f5_push = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
                   for p in push_pp_res]
        means_f5_push.append(stat.mean(f5_push))
        stds_f5_push.append(stat.stdev(f5_push))
        f5_push_modif = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
                         for p in push_pp_res_modif]
        means_f5_push_modif.append(stat.mean(f5_push_modif))
        stds_f5_push_modif.append(stat.stdev(f5_push_modif))
        #for f_4
        f4_push = [np.sum(f_4(p.points))/p.points.shape[0] 
                   for p in push_pp_res]
        means_f4_push.append(stat.mean(f4_push))
        stds_f4_push.append(stat.stdev(f4_push))
        f4_push_modif = [np.sum(f_4(p.points))/p.points.shape[0] 
                         for p in push_pp_res_modif]
        means_f4_push_modif.append(stat.mean(f4_push_modif))
        stds_f4_push_modif.append(stat.stdev(f4_push_modif))
        #for f_2
        f2_push = [np.sum(f_2(p.points))/p.points.shape[0] 
                   for p in push_pp_res]
        means_f2_push.append(stat.mean(f2_push))
        stds_f2_push.append(stat.stdev(f2_push))
        f2_push_modif = [np.sum(f_2(p.points))/p.points.shape[0] 
                         for p in push_pp_res_modif]
        means_f2_push_modif.append(stat.mean(f2_push_modif))
        stds_f2_push_modif.append(stat.stdev(f2_push_modif))
        #for f_1
        f1_push = [np.sum(f_1(p.points))/p.points.shape[0] 
                   for p in push_pp_res]
        means_f1_push.append(stat.mean(f1_push))
        stds_f1_push.append(stat.stdev(f1_push))
        f1_push_modif = [np.sum(f_1(p.points))/p.points.shape[0] 
                         for p in push_pp_res_modif]
        means_f1_push_modif.append(stat.mean(f1_push_modif))
        stds_f1_push_modif.append(stat.stdev(f1_push_modif))
        print("std poisson f_5", std_f5_poisson, "std push", stds_f5_push,
              "std push modif", stds_f5_push_modif)
        print("std poisson f_4", std_f4_poisson, "std push", stds_f4_push,
              "std push modif", stds_f4_push_modif)
        print("std poisson f_2", std_f2_poisson, "std push", stds_f2_push,
              "std push modif", stds_f2_push_modif)
        print("std poisson f_1", std_f1_poisson, "std push", stds_f1_push,
              "std push modif", stds_f1_push_modif)
        print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_strat)

In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points, R=3)
z_2 = f_4(np.array([x_2, x_2]).T)
z_3 = f_2(np.array([x_2, x_2]).T)
z_4 = f_1(np.array([x_2, x_2]).T)
fig = plt.figure(figsize=(17, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_3, c=z_3)
ax.set_title("f_3")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title("f_4")
plt.savefig("func_for_push_modif.pdf")
plt.show()
fig, ax = plt.subplots(1,4, figsize=(20, 3))
ax[0].plot(epsilons, stds_f5_push, "k.", label="push" )
ax[0].plot(epsilons, stds_f5_push, "k" )
ax[0].plot(epsilons, stds_f5_push_modif, "b.", label="push modif" )
ax[0].plot(epsilons, stds_f5_push_modif, "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons, stds_f4_push, "k.", label="push" )
ax[1].plot(epsilons, stds_f4_push, "k" )
ax[1].plot(epsilons, stds_f4_push_modif, "b.", label="push modif" )
ax[1].plot(epsilons, stds_f4_push_modif, "b" )
ax[1].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons, stds_f2_push, "k.", label="push" )
ax[2].plot(epsilons, stds_f2_push, "k" )
ax[2].plot(epsilons, stds_f2_push_modif, "b.", label="push modif" )
ax[2].plot(epsilons, stds_f2_push_modif, "b" )
ax[2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons, stds_f1_push, "k.", label="push" )
ax[3].plot(epsilons, stds_f1_push, "k" )
ax[3].plot(epsilons, stds_f1_push_modif, "b.", label="push modif" )
ax[3].plot(epsilons, stds_f1_push_modif, "b" )
ax[3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("Variance f_4")
plt.savefig("std_push_modif.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(20, 8))
ax[0, 0].plot(epsilons, stds_f5_push, "k.", label="push" )
ax[0, 0].plot(epsilons, stds_f5_push, "k" )
ax[0, 0].plot(0, std_f5_poisson, "r.", label="Poisson")
ax[1, 0].plot(epsilons, stds_f5_push_modif, "b.", label="push modif" )
ax[1, 0].plot(epsilons, stds_f5_push_modif, "b" )
ax[1, 0].plot(0, std_f5_poisson, "r.", label="Poisson")
#ax[0].set_title("Variance bump function")
ax[1, 0].legend()
ax[1, 0].set_xlabel(r"$\epsilon$")
ax[1, 0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1, 0].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[0, 1].plot(epsilons, stds_f4_push, "k.", label="push" )
ax[0, 1].plot(epsilons, stds_f4_push, "k" )
ax[1, 1].plot(epsilons, stds_f4_push_modif, "b.", label="push modif" )
ax[1, 1].plot(epsilons, stds_f4_push_modif, "b" )
ax[1, 1].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[1, 1].legend()
ax[1, 1].set_xlabel(r"$\epsilon$")
ax[0, 2].plot(epsilons, stds_f2_push, "k.", label="push" )
ax[0, 2].plot(epsilons, stds_f2_push, "k" )
ax[0, 2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[1, 2].plot(epsilons, stds_f2_push_modif, "b.", label="push modif" )
ax[1, 2].plot(epsilons, stds_f2_push_modif, "b" )
ax[1, 2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[1, 2].legend()
ax[1, 2].set_xlabel(r"$\epsilon$")
ax[0, 3].plot(epsilons, stds_f1_push, "k.", label="push" )
ax[0, 3].plot(epsilons, stds_f1_push, "k" )
ax[0, 3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[1, 3].plot(epsilons, stds_f1_push_modif, "b.", label="push modif" )
ax[1, 3].plot(epsilons, stds_f1_push_modif, "b" )
ax[1, 3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[1, 3].legend()
ax[1, 3].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("Variance f_4")
#plt.savefig("std_push_modif.pdf")
plt.show()

# 6- Tests for Equi pp 

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

In [ ]:
d = 3
simu_window = BallWindow(center=[0]*d, radius=6.5)
window = BallWindow(center=[0]*d, radius=3)
intensity = 6.2
samples_nb = 20
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
n=10
f5_poisson = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in poisson_pp_res]
mean_f5_poisson = stat.mean(f5_poisson)
std_f5_poisson = stat.stdev(f5_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)

In [ ]:
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = [-epsilon_th, -0.004, -0.001, -0.0005, -0.0001,0,
            0.0001, 0.0005, 0.001, 0.004, 0.005, epsilon_th]
means_f5_push, stds_f5_push, means_f5_equi, stds_f5_equi = [], [], [], []
means_f4_push, stds_f4_push, means_f4_equi, stds_f4_equi = [], [], [], []
means_f2_push, stds_f2_push, means_f2_equi, stds_f2_equi = [], [], [], []
means_f1_push, stds_f1_push, means_f1_equi, stds_f1_equi = [], [], [], []
time_start = time.time() 
print("start_time",  time.time() - time_start)
print("----------------------------------------------")
push_pp_res_list=[]
if __name__ == "__main__":
    for g in gpp_pp: 
        push_pp = g.pushed_point_pattern(epsilon=epsilons, stop_time=1) 
        push_pp_res_ = np.array([p.restrict_to_window(window) for p in push_pp])
        push_pp_res_list.append(push_pp_res_)
print("time push", time.time() - time_start)
for e in range(len(epsilons)):
    print("epsilon", epsilons[e])
    gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
    #equi 
    equi_pp = [g.equilibrium_point_pattern(epsilon=epsilons[e], stop_time=1) for g in gpp_pp]
    equi_pp_res = [p.restrict_to_window(window) for p in equi_pp]
    #push
    push_pp_res = [push_pp_res_list[i][e] for i in range(samples_nb)]
    # mean number of points in the restricted window
    pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
    push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
    equi_res_pt_nb = stat.mean(equi_pp_res[i].points.shape[0] for i in range(samples_nb))

    print("nb_pts_poi", pois_res_pt_nb, 
          "nb_pts_push", push_res_pt_nb,
         "nb_pts_equi", equi_res_pt_nb,)
    #for f_5
    f5_push = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
               for p in push_pp_res]
    means_f5_push.append(stat.mean(f5_push))
    stds_f5_push.append(stat.stdev(f5_push))
    f5_equi = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
                     for p in equi_pp_res]
    means_f5_equi.append(stat.mean(f5_equi))
    stds_f5_equi.append(stat.stdev(f5_equi))
    #for f_4
    f4_push = [np.sum(f_4(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f4_push.append(stat.mean(f4_push))
    stds_f4_push.append(stat.stdev(f4_push))
    f4_equi = [np.sum(f_4(p.points))/p.points.shape[0] 
                     for p in equi_pp_res]
    means_f4_equi.append(stat.mean(f4_equi))
    stds_f4_equi.append(stat.stdev(f4_equi))
    #for f_2
    f2_push = [np.sum(f_2(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f2_push.append(stat.mean(f2_push))
    stds_f2_push.append(stat.stdev(f2_push))
    f2_equi = [np.sum(f_2(p.points))/p.points.shape[0] 
                     for p in equi_pp_res]
    means_f2_equi.append(stat.mean(f2_equi))
    stds_f2_equi.append(stat.stdev(f2_equi))
    #for f_1
    f1_push = [np.sum(f_1(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f1_push.append(stat.mean(f1_push))
    stds_f1_push.append(stat.stdev(f1_push))
    f1_equi = [np.sum(f_1(p.points))/p.points.shape[0] 
                     for p in equi_pp_res]
    means_f1_equi.append(stat.mean(f1_equi))
    stds_f1_equi.append(stat.stdev(f1_equi))
    print("std poisson f_5=", std_f5_poisson, ", std push=", stds_f5_push,
          ", std equi=", stds_f5_equi)
    print("std poisson f_4=", std_f4_poisson, ", std push=", stds_f4_push,
          ", std equi=", stds_f4_equi)
    print("std poisson f_2=", std_f2_poisson, ", std push=", stds_f2_push,
          ", std equi=", stds_f2_equi)
    print("std poisson f_1=", std_f1_poisson, ", std push=", stds_f1_push,
          ", std equi=", stds_f1_equi)
    print("----------------------------------------------")
time_end = time.time()
print("time: ", time_end - time_start)

In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points, R=3)
z_2 = f_4(np.array([x_2, x_2]).T)
z_3 = f_2(np.array([x_2, x_2]).T)
z_4 = f_1(np.array([x_2, x_2]).T)
fig = plt.figure(figsize=(17, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_3, c=z_3)
ax.set_title("f_3")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title("f_4")
plt.savefig("func_for_push_modif.pdf")
plt.show()
fig, ax = plt.subplots(1,4, figsize=(20, 6))
ax[0].plot(epsilons[:-2], stds_f5_push[:-2], "k.", label="push" )
ax[0].plot(epsilons[:-2], stds_f5_push[:-2], "k" )
ax[0].plot(epsilons[:-2], stds_f5_equi[:-2], "b.", label="equi" )
ax[0].plot(epsilons[:-2], stds_f5_equi[:-2], "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons[:-2], stds_f4_push[:-2], "k.", label="push" )
ax[1].plot(epsilons[:-2], stds_f4_push[:-2], "k" )
ax[1].plot(epsilons[:-2], stds_f4_equi[:-2], "b.", label="equi" )
ax[1].plot(epsilons[:-2], stds_f4_equi[:-2], "b" )
ax[1].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons[:-1], stds_f2_push[:-1], "k.", label="push" )
ax[2].plot(epsilons[:-1], stds_f2_push[:-1], "k" )
ax[2].plot(epsilons[:-1], stds_f2_equi[:-1], "b.", label="equi" )
ax[2].plot(epsilons[:-1], stds_f2_equi[:-1], "b" )
ax[2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons[:-1], stds_f1_push[:-1], "k.", label="push" )
ax[3].plot(epsilons[:-1], stds_f1_push[:-1], "k" )
ax[3].plot(epsilons[:-1], stds_f1_equi[:-1], "b.", label="equi" )
ax[3].plot(epsilons[:-1], stds_f1_equi[:-1], "b" )
ax[3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("Variance f_4")
#plt.savefig("std_push_and_equi.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20, 6))
ax[0].plot(epsilons[:-2], stds_f5_push[:-2], "k.", label="push" )
ax[0].plot(epsilons[:-2], stds_f5_push[:-2], "k" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons[:-2], stds_f4_push[:-2], "k.", label="push" )
ax[1].plot(epsilons[:-2], stds_f4_push[:-2], "k" )
ax[1].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons[:-1], stds_f2_push[:-1], "k.", label="push" )
ax[2].plot(epsilons[:-1], stds_f2_push[:-1], "k" )
ax[2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons[:-1], stds_f1_push[:-1], "k.", label="push" )
ax[3].plot(epsilons[:-1], stds_f1_push[:-1], "k" )
ax[3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("Variance f_4")
#plt.savefig("std_push_and_equi.pdf")
plt.show()
fig, ax = plt.subplots(1,4, figsize=(20, 6))
ax[0].plot(epsilons[:-2], stds_f5_equi[:-2], "b.", label="equi" )
ax[0].plot(epsilons[:-2], stds_f5_equi[:-2], "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons[:-2], stds_f4_equi[:-2], "b.", label="equi" )
ax[1].plot(epsilons[:-2], stds_f4_equi[:-2], "b" )
ax[1].plot(0, std_f4_poisson, "r.", label="Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons[:-1], stds_f2_equi[:-1], "b.", label="equi" )
ax[2].plot(epsilons[:-1], stds_f2_equi[:-1], "b" )
ax[2].plot(0, std_f2_poisson, "r.", label="Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons[:-1], stds_f1_equi[:-1], "b.", label="equi" )
ax[3].plot(epsilons[:-1], stds_f1_equi[:-1], "b" )
ax[3].plot(0, std_f1_poisson, "r.", label="Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[1].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
#ax[1].set_title("Variance f_4")
#plt.savefig("std_push_and_equi.pdf")
plt.show()

In [ ]:
list[np.linspace(-epsilon_th, epsilon_th, 30)]

# 7- Use a truncated version of the force
 $$G^{p,q}(x) = \sum_{z \in \mathcal{P}} \frac{z - x}{\|z-x\|_2^d} \mathbf{1}_{B(x,p) \setminus B(x, q)}(z)
 $$

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

In [ ]:
d = 3
simu_window = BallWindow(center=[0]*d, radius=15)
window = BallWindow(center=[0]*d, radius=5)
intensity = 3
samples_nb = 50
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
n=10
f5_poisson = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in poisson_pp_res]
mean_f5_poisson = stat.mean(f5_poisson)
std_f5_poisson = stat.stdev(f5_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)
#indicator fct
f3_poisson = [p.points.shape[0] for p in poisson_pp_res]
mean_f3_poisson = stat.mean(f3_poisson)
std_f3_poisson = stat.stdev(f3_poisson)
print("mean_poisson/V", mean_f3_poisson/window.volume, ",intensity",intensity )

In [ ]:
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th)
epsilons = np.linspace(-0.003, 0.003, 100).tolist()
print_time=True
print("----------------------------------------------")
push_pp_res_list=[]
push_pp_trunc_res_list=[]
t=0
p=6
q=0.1
if __name__ == "__main__":
    for g in gpp_pp: 
        t = t+1
        print(t)
        #push trunc
        time_start = time.time()
        push_pp_trunc = g.pushed_point_pattern(epsilon=epsilons, stop_time=1, force_truncated=True, p=p, q=q) 
        push_pp_trunc_res_ = np.array([p.restrict_to_window(window) for p in push_pp_trunc])
        push_pp_trunc_res_list.append(push_pp_trunc_res_)
        if print_time:
            print("start_time",  time.time() - time_start)
            
        #push
        time_start = time.time()
        push_pp = g.pushed_point_pattern(epsilon=epsilons, stop_time=1) 
        push_pp_res_ = np.array([p.restrict_to_window(window) for p in push_pp])
        push_pp_res_list.append(push_pp_res_)
        if print_time:
            print("start_time",  time.time() - time_start)
            print_time=False
        
print("time push", time.time() - time_start)


In [ ]:
push_pp_res_list[0].shape

In [ ]:
means_f5_push, stds_f5_push, means_f5_push_trunc, stds_f5_push_trunc = [], [], [], []
means_f4_push, stds_f4_push, means_f4_push_trunc, stds_f4_push_trunc = [], [], [], []
means_f3_push, stds_f3_push, means_f3_push_trunc, stds_f3_push_trunc = [], [], [], []
means_f2_push, stds_f2_push, means_f2_push_trunc, stds_f2_push_trunc = [], [], [], []
means_f1_push, stds_f1_push, means_f1_push_trunc, stds_f1_push_trunc = [], [], [], []
for e in range(len(epsilons)):
    print("epsilon", epsilons[e])
    #push
    push_pp_res = [push_pp_res_list[i][e] for i in range(samples_nb)]
    push_pp_trunc_res = [push_pp_trunc_res_list[i][e] for i in range(samples_nb)]
    # mean number of points in the restricted window
    pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
    push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
    push_res_trunc_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
    
    

    print("nb_pts_poi", pois_res_pt_nb, 
          "nb_pts_push", push_res_pt_nb)
    #for f_5
    f5_push = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
               for p in push_pp_res]
    means_f5_push.append(stat.mean(f5_push))
    stds_f5_push.append(stat.stdev(f5_push))
    f5_push_trunc = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f5_push_trunc.append(stat.mean(f5_push_trunc))
    stds_f5_push_trunc.append(stat.stdev(f5_push_trunc))
    #for f_4
    f4_push = [np.sum(f_4(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f4_push.append(stat.mean(f4_push))
    stds_f4_push.append(stat.stdev(f4_push))
    f4_push_trunc = [np.sum(f_4(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f4_push_trunc.append(stat.mean(f4_push_trunc))
    stds_f4_push_trunc.append(stat.stdev(f4_push_trunc))
    #for f_3 (indicator fct)
    f3_push = [p.points.shape[0] for p in push_pp_res]
    means_f3_push.append(stat.mean(f3_push))
    stds_f3_push.append(stat.stdev(f3_push))
    f3_push_trunc = [p.points.shape[0] for p in push_pp_trunc_res]
    means_f3_push_trunc.append(stat.mean(f3_push_trunc))
    stds_f3_push_trunc.append(stat.stdev(f3_push_trunc))
    #for f_2
    f2_push = [np.sum(f_2(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f2_push.append(stat.mean(f2_push))
    stds_f2_push.append(stat.stdev(f2_push))
    f2_push_trunc = [np.sum(f_2(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f2_push_trunc.append(stat.mean(f2_push_trunc))
    stds_f2_push_trunc.append(stat.stdev(f2_push_trunc))
    #for f_1
    f1_push = [np.sum(f_1(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f1_push.append(stat.mean(f1_push))
    stds_f1_push.append(stat.stdev(f1_push))
    f1_push_trunc = [np.sum(f_1(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f1_push_trunc.append(stat.mean(f1_push_trunc))
    stds_f1_push_trunc.append(stat.stdev(f1_push_trunc))
    #print("std poisson f_5=", std_f5_poisson, ", std push=", stds_f5_push, ", std push trunc=", stds_f5_push_trunc)
    #print("std poisson f_4=", std_f4_poisson, ", std push=", stds_f4_push, ", std push trunc=", stds_f4_push_trunc)
    #print("std poisson f_2=", std_f2_poisson, ", std push=", stds_f2_push, ", std push trunc=", stds_f2_push_trunc)
    #print("std poisson f_1=", std_f1_poisson, ", std push=", stds_f1_push, ", std push trunc=", stds_f1_push_trunc)
    print("----------------------------------------------")


In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_5 = bump_indicator(10, points, R=3)
z_4 = f_4(np.array([x_2, x_2]).T)
z_2 = f_2(np.array([x_2, x_2]).T)
z_1 = f_1(np.array([x_2, x_2]).T)
z_3 = f_3(np.array([x_2, x_2]).T, window)

fig = plt.figure(figsize=(17, 5))
ax = fig.add_subplot(1, 5, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_5, c=z_5)
ax.set_title("Bump fct")
ax = fig.add_subplot(1, 5, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
ax.set_title("indicator fct")
ax = fig.add_subplot(1, 5, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_1, c=z_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 5, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 5, 5, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title("f_4")
#plt.savefig("func_for_push_modif.pdf")
plt.show()

fig, ax = plt.subplots(1,5, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push, "k.", label="Push" )
ax[0].plot(epsilons, stds_f5_push, "k" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b.", label="Push trun" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons, stds_f3_push, "k.", label="Push" )
ax[1].plot(epsilons, stds_f3_push, "k" )
ax[1].plot(epsilons, stds_f3_push_trunc, "b.", label="Push trun" )
ax[1].plot(epsilons, stds_f3_push_trunc, "b" )
ax[1].plot(0, std_f3_poisson, "r.", label="Poisson")
ax[1].set_title("Variance indicator function")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")

ax[2].plot(epsilons, stds_f1_push, "k.", label=" Push" )
ax[2].plot(epsilons, stds_f1_push, "k" )
ax[2].plot(epsilons, stds_f1_push_trunc, "b.", label="Push trunc" )
ax[2].plot(epsilons, stds_f1_push_trunc, "b" )
ax[2].plot(0, std_f1_poisson, "r.", label=" Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[2].set_title("Variance f_1")

ax[3].plot(epsilons, stds_f2_push, "k.", label=" Push" )
ax[3].plot(epsilons, stds_f2_push, "k" )
ax[3].plot(epsilons, stds_f2_push_trunc, "b.", label="Push trunc" )
ax[3].plot(epsilons, stds_f2_push_trunc, "b" )
ax[3].plot(0, std_f2_poisson, "r.", label=" Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
ax[3].set_title("Variance f_2")

ax[4].plot(epsilons, stds_f4_push, "k.", label=" Push" )
ax[4].plot(epsilons, stds_f4_push, "k" )
ax[4].plot(epsilons, stds_f4_push_trunc, "b.", label="Push trunc" )
ax[4].plot(epsilons, stds_f4_push_trunc, "b" )
ax[4].plot(0, std_f4_poisson, "r.", label=" Poisson")
ax[4].legend()
ax[4].set_xlabel(r"$\epsilon$")
ax[4].set_title("Variance f_4")
plt.show()




In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_1 = bump_indicator(10, points, R=3)
z_2 = f_4(np.array([x_2, x_2]).T)
z_3 = f_2(np.array([x_2, x_2]).T)
z_4 = f_1(np.array([x_2, x_2]).T)
fig = plt.figure(figsize=(28, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_1, c=z_1)
ax.set_title("f_1")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title("f_2")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_3, c=z_3)
ax.set_title("f_3")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title("f_4")
plt.savefig("func_for_push_modif.pdf")
plt.show()

fig, ax = plt.subplots(1,4, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push, "k")
ax[0].scatter(epsilons, stds_f5_push, c="k", s=0.1, label="f_1 Push" )
ax[0].plot(0, std_f5_poisson, "r.", label="f_1 Poisson")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons, stds_f4_push, "k")
ax[1].scatter(epsilons, stds_f4_push, c="k", s=0.1, label="f_2 Push" )
ax[1].plot(0, std_f4_poisson, "r.", label="f_2 Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons, stds_f2_push, "k")
ax[2].scatter(epsilons, stds_f2_push, c="k", s=0.1, label="f_3 Push" )
ax[2].plot(0, std_f2_poisson, "r.", label="f_3 Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons, stds_f1_push, "k")
ax[3].scatter(epsilons, stds_f1_push, c="k", s=0.1, label="f_4 Push" )
ax[3].plot(0, std_f1_poisson, "r.", label="f_4 Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
plt.savefig("std_push1_wrt_epsilon.pdf")
plt.show()

fig, ax = plt.subplots(1,4, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push_trunc, "b", label="f_1 Push trun" )
ax[0].plot(0, std_f5_poisson, "r.", label="f_1 Poisson")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")
ax[1].plot(epsilons, stds_f4_push_trunc, "b", label="f_2 Push trunc" )
ax[1].plot(0, std_f4_poisson, "r.", label="f_2 Poisson")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
ax[2].plot(epsilons, stds_f2_push_trunc, "b", label="f_3 Push trunc" )
ax[2].plot(0, std_f2_poisson, "r.", label="f_3 Poisson")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
ax[3].plot(epsilons, stds_f1_push_trunc, "b", label="f_4 Push trunc" )
ax[3].plot(0, std_f1_poisson, "r.", label="f_4 Poisson")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
plt.savefig("std_trunc1_wrt_epsilon.pdf")
plt.show()



# 8- Test if $\epsilon= 1/(2\rho \kappa_d d)$ is a spetial value of epsilon

In [ ]:
d = 3
simu_window = BallWindow(center=[0]*d, radius=15)
window = BallWindow(center=[0]*d, radius=5)
intensity = 3
samples_nb = 50
mean_points_nb = intensity*simu_window.volume 
print("points_nb=", mean_points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = [poisson.generate_point_pattern(window=simu_window) for _ in range(samples_nb)]
poisson_pp_res = [p.restrict_to_window(window) for p in poisson_pp]
# mean and var for poisson
n=10
f5_poisson = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] for p in poisson_pp_res]
mean_f5_poisson = stat.mean(f5_poisson)
std_f5_poisson = stat.stdev(f5_poisson)
f4_poisson = [np.sum(f_4(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f4_poisson = stat.mean(f4_poisson)
std_f4_poisson = stat.stdev(f4_poisson)
f2_poisson = [np.sum(f_2(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f2_poisson = stat.mean(f2_poisson)
std_f2_poisson = stat.stdev(f2_poisson)
f1_poisson = [np.sum(f_1(p.points))/p.points.shape[0] for p in poisson_pp_res]
mean_f1_poisson = stat.mean(f1_poisson)
std_f1_poisson = stat.stdev(f1_poisson)
#indicator fct
f3_poisson = [p.points.shape[0] for p in poisson_pp_res]
mean_f3_poisson = stat.mean(f3_poisson)
std_f3_poisson = stat.stdev(f3_poisson)
print("mean_poisson/V", mean_f3_poisson/window.volume, ",intensity",intensity )

In [ ]:
from structure_factor.spatial_windows import UnitBallWindow
window_unit = UnitBallWindow(center = [0,0,0])
kappa_d = window_unit.volume
epsilon_special = 1/(2*poisson_pp[0].intensity*d*kappa_d)
#push
gpp_pp = [GravityPointProcess(p) for p in poisson_pp]
epsilon_th = gpp_pp[0].epsilon
print("epsilon_th", epsilon_th, "spetial epsilon=", epsilon_special)
epsilons = np.linspace(-0.1, 0.1, 100).tolist()
epsilons.append(epsilon_special)
epsilons.append(epsilon_th)
epsilons.sort()
#epsilons

In [ ]:

print_time=True
print("----------------------------------------------")
push_pp_res_list=[]
push_pp_trunc_res_list=[]
t=0
p=6
q=0.1
time_0 = time.time()
if __name__ == "__main__":
    for g in gpp_pp: 
        t = t+1
        print(t)
        #push trunc
        time_start = time.time()
        push_pp_trunc = g.pushed_point_pattern(epsilon=epsilons, stop_time=1, force_truncated=True, p=p, q=q) 
        push_pp_trunc_res_ = np.array([p.restrict_to_window(window) for p in push_pp_trunc])
        push_pp_trunc_res_list.append(push_pp_trunc_res_)
        if print_time:
            print("start_time",  time.time() - time_start)
            
        #push
        time_start = time.time()
        push_pp = g.pushed_point_pattern(epsilon=epsilons, stop_time=1) 
        push_pp_res_ = np.array([p.restrict_to_window(window) for p in push_pp])
        push_pp_res_list.append(push_pp_res_)
        if print_time:
            print("start_time",  time.time() - time_start)
            print_time=False
        
print("time push", time.time() - time_0)


In [ ]:
means_f5_push, stds_f5_push, means_f5_push_trunc, stds_f5_push_trunc = [], [], [], []
means_f4_push, stds_f4_push, means_f4_push_trunc, stds_f4_push_trunc = [], [], [], []
means_f3_push, stds_f3_push, means_f3_push_trunc, stds_f3_push_trunc = [], [], [], []
means_f2_push, stds_f2_push, means_f2_push_trunc, stds_f2_push_trunc = [], [], [], []
means_f1_push, stds_f1_push, means_f1_push_trunc, stds_f1_push_trunc = [], [], [], []
for e in range(len(epsilons)):
    print("epsilon", epsilons[e])
    #push
    push_pp_res = [push_pp_res_list[i][e] for i in range(samples_nb)]
    push_pp_trunc_res = [push_pp_trunc_res_list[i][e] for i in range(samples_nb)]
    # mean number of points in the restricted window
    pois_res_pt_nb = stat.mean(poisson_pp_res[i].points.shape[0] for i in range(samples_nb))
    push_res_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
    push_res_trunc_pt_nb = stat.mean(push_pp_res[i].points.shape[0] for i in range(samples_nb))
    
    

    print("nb_pts_poi", pois_res_pt_nb, 
          "nb_pts_push", push_res_pt_nb)
    #for f_5
    f5_push = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
               for p in push_pp_res]
    means_f5_push.append(stat.mean(f5_push))
    stds_f5_push.append(stat.stdev(f5_push))
    f5_push_trunc = [np.sum(bump_indicator(n, p.points, R=3))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f5_push_trunc.append(stat.mean(f5_push_trunc))
    stds_f5_push_trunc.append(stat.stdev(f5_push_trunc))
    #for f_4
    f4_push = [np.sum(f_4(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f4_push.append(stat.mean(f4_push))
    stds_f4_push.append(stat.stdev(f4_push))
    f4_push_trunc = [np.sum(f_4(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f4_push_trunc.append(stat.mean(f4_push_trunc))
    stds_f4_push_trunc.append(stat.stdev(f4_push_trunc))
    #for f_3 (indicator fct)
    f3_push = [p.points.shape[0] for p in push_pp_res]
    means_f3_push.append(stat.mean(f3_push))
    stds_f3_push.append(stat.stdev(f3_push))
    f3_push_trunc = [p.points.shape[0] for p in push_pp_trunc_res]
    means_f3_push_trunc.append(stat.mean(f3_push_trunc))
    stds_f3_push_trunc.append(stat.stdev(f3_push_trunc))
    #for f_2
    f2_push = [np.sum(f_2(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f2_push.append(stat.mean(f2_push))
    stds_f2_push.append(stat.stdev(f2_push))
    f2_push_trunc = [np.sum(f_2(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f2_push_trunc.append(stat.mean(f2_push_trunc))
    stds_f2_push_trunc.append(stat.stdev(f2_push_trunc))
    #for f_1
    f1_push = [np.sum(f_1(p.points))/p.points.shape[0] 
               for p in push_pp_res]
    means_f1_push.append(stat.mean(f1_push))
    stds_f1_push.append(stat.stdev(f1_push))
    f1_push_trunc = [np.sum(f_1(p.points))/p.points.shape[0] 
               for p in push_pp_trunc_res]
    means_f1_push_trunc.append(stat.mean(f1_push_trunc))
    stds_f1_push_trunc.append(stat.stdev(f1_push_trunc))
    #print("std poisson f_5=", std_f5_poisson, ", std push=", stds_f5_push, ", std push trunc=", stds_f5_push_trunc)
    #print("std poisson f_4=", std_f4_poisson, ", std push=", stds_f4_push, ", std push trunc=", stds_f4_push_trunc)
    #print("std poisson f_2=", std_f2_poisson, ", std push=", stds_f2_push, ", std push trunc=", stds_f2_push_trunc)
    #print("std poisson f_1=", std_f1_poisson, ", std push=", stds_f1_push, ", std push trunc=", stds_f1_push_trunc)
    print("----------------------------------------------")


In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_5 = bump_indicator(10, points, R=3)
z_4 = f_4(np.array([x_2, x_2]).T)
z_2 = f_2(np.array([x_2, x_2]).T)
z_1 = f_1(np.array([x_2, x_2]).T)
z_3 = f_3(points, BallWindow(center=[0,0], radius=window.radius))

fig = plt.figure(figsize=(14, 5))
ax = fig.add_subplot(1, 5, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_5, c=z_5)
ax.set_title("Bump function")
ax = fig.add_subplot(1, 5, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
ax.set_title("indicator function")
ax = fig.add_subplot(1, 5, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_1, c=z_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 5, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 5, 5, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title(r"$f_3$")
plt.savefig("func_for_push_with_epsilon_zero.pdf")
plt.show()

fig, ax = plt.subplots(1,5, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push, "k.", label="Push" )
ax[0].plot(epsilons, stds_f5_push, "k" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b.", label="Push trun" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
ax[0].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")

ax[1].plot(epsilons, stds_f3_push, "k.", label="F" )
ax[1].plot(epsilons, stds_f3_push, "k" )
ax[1].plot(epsilons, stds_f3_push_trunc, "b.", label="F trunc" )
ax[1].plot(epsilons, stds_f3_push_trunc, "b" )
ax[1].plot(0, std_f3_poisson, "r.", label="Poisson")
ax[1].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
#ax[1].set_title("Variance indicator function")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")

ax[2].plot(epsilons, stds_f1_push, "k.", label="F" )
ax[2].plot(epsilons, stds_f1_push, "k" )
ax[2].plot(epsilons, stds_f1_push_trunc, "b.", label="F trunc" )
ax[2].plot(epsilons, stds_f1_push_trunc, "b" )
ax[2].plot(0, std_f1_poisson, "r.", label=" Poisson")
ax[2].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
#ax[2].set_title("Variance f_1")

ax[3].plot(epsilons, stds_f2_push, "k.", label="F" )
ax[3].plot(epsilons, stds_f2_push, "k" )
ax[3].plot(epsilons, stds_f2_push_trunc, "b.", label="F trunc" )
ax[3].plot(epsilons, stds_f2_push_trunc, "b" )
ax[3].plot(0, std_f2_poisson, "r.", label=" Poisson")
ax[3].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[3].set_title("Variance f_2")

ax[4].plot(epsilons, stds_f4_push, "k.", label="F" )
ax[4].plot(epsilons, stds_f4_push, "k" )
ax[4].plot(epsilons, stds_f4_push_trunc, "b.", label="F trunc" )
ax[4].plot(epsilons, stds_f4_push_trunc, "b" )
ax[4].plot(0, std_f4_poisson, "r.", label=" Poisson")
ax[4].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[4].legend()
ax[4].set_xlabel(r"$\epsilon$")
#ax[4].set_title("Variance f_4")
plt.savefig("std_push_with_epsilon_0.pdf")
plt.show()




In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_3 = f_3(points, BallWindow(center=[0,0], radius=window.radius))

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_3, c=z_3)
plt.savefig("indicator_fct.pdf")
plt.show()

fig, ax = plt.subplots( figsize=(7, 5))
ax.plot(epsilons, stds_f3_push, "k.", label="F" )
ax.plot(epsilons, stds_f3_push, "k" )
ax.plot(epsilons, stds_f3_push_trunc, "b.", label=r"$G^{(p,q)}$" )
ax.plot(epsilons, stds_f3_push_trunc, "b" )
ax.plot(0, std_f3_poisson, "r.", label="Poisson")
ax.axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax.set_ylabel(r"$\sigma(\sum_{x} \mathbf{1}_{B(0,R)(x)})$ ")
ax.legend()
ax.set_xlabel(r"$\epsilon$")
plt.savefig("std_indicator_with_epsilon_0.pdf")
plt.show()




In [ ]:
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_5 = bump_indicator(10, points, R=3)
z_4 = f_4(np.array([x_2, x_2]).T)
z_2 = f_2(np.array([x_2, x_2]).T)
z_1 = f_1(np.array([x_2, x_2]).T)
z_3 = f_3(points, BallWindow(center=[0,0], radius=window.radius))

fig = plt.figure(figsize=(14, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_5, c=z_5)
ax.set_title("Bump function")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(x_2, x_2, z_1, c=z_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title(r"$f_3$")
plt.savefig("func_for_push_with_epsilon_zero.pdf")
plt.show()

fig, ax = plt.subplots(1,4, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push, "k.", label="Push" )
ax[0].plot(epsilons, stds_f5_push, "k" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b.", label="Push trun" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
ax[0].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")

ax[1].plot(epsilons, stds_f1_push, "k.", label="F" )
ax[1].plot(epsilons, stds_f1_push, "k" )
ax[1].plot(epsilons, stds_f1_push_trunc, "b.", label="F trunc" )
ax[1].plot(epsilons, stds_f1_push_trunc, "b" )
ax[1].plot(0, std_f1_poisson, "r.", label=" Poisson")
ax[1].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
#ax[2].set_title("Variance f_1")

ax[2].plot(epsilons, stds_f2_push, "k.", label="F" )
ax[2].plot(epsilons, stds_f2_push, "k" )
ax[2].plot(epsilons, stds_f2_push_trunc, "b.", label="F trunc" )
ax[2].plot(epsilons, stds_f2_push_trunc, "b" )
ax[2].plot(0, std_f2_poisson, "r.", label=" Poisson")
ax[2].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
#ax[3].set_title("Variance f_2")

ax[3].plot(epsilons, stds_f4_push, "k.", label="F" )
ax[3].plot(epsilons, stds_f4_push, "k" )
ax[3].plot(epsilons, stds_f4_push_trunc, "b.", label="F trunc" )
ax[3].plot(epsilons, stds_f4_push_trunc, "b" )
ax[3].plot(0, std_f4_poisson, "r.", label=" Poisson")
ax[3].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[4].set_title("Variance f_4")
plt.savefig("std_push_with_epsilon_0.pdf")
plt.show()




In [ ]:
# with epsilon th
x = np.linspace(-10, 10, 100)
x_2 = np.linspace(-10, 10, 500)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_5 = bump_indicator(10, points, R=3)
z_4 = f_4(np.array([x_2, x_2]).T)
z_2 = f_2(np.array([x_2, x_2]).T)
z_1 = f_1(np.array([x_2, x_2]).T)
z_3 = f_3(points, BallWindow(center=[0,0], radius=window.radius))

fig = plt.figure(figsize=(14, 5))
ax = fig.add_subplot(1, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_5, c=z_5)
ax.set_title("Bump function")
ax = fig.add_subplot(1, 4, 2, projection='3d')
ax.scatter3D(x_2, x_2, z_1, c=z_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(1, 4, 3, projection='3d')
ax.scatter3D(x_2, x_2, z_2, c=z_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(1, 4, 4, projection='3d')
ax.scatter3D(x_2, x_2, z_4, c=z_4)
ax.set_title(r"$f_3$")
plt.show()

fig, ax = plt.subplots(1,4, figsize=(25, 5))
ax[0].plot(epsilons, stds_f5_push, "k.", label="Push" )
ax[0].plot(epsilons, stds_f5_push, "k" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b.", label="Push trun" )
ax[0].plot(epsilons, stds_f5_push_trunc, "b" )
ax[0].plot(0, std_f5_poisson, "r.", label="Poisson")
ax[0].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[0].axvline(epsilon_th, color="m", linestyle="--", label=r"$\epsilon_{th}$")
#ax[0].set_title("Variance bump function")
ax[0].legend()
ax[0].set_xlabel(r"$\epsilon$")
ax[0].set_ylabel(r"$\sigma(\sum_{x} f(x)/N)$ ")

ax[1].plot(epsilons, stds_f1_push, "k.", label="F" )
ax[1].plot(epsilons, stds_f1_push, "k" )
ax[1].plot(epsilons, stds_f1_push_trunc, "b.", label="F trunc" )
ax[1].plot(epsilons, stds_f1_push_trunc, "b" )
ax[1].plot(0, std_f1_poisson, "r.", label=" Poisson")
ax[1].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[1].axvline(epsilon_th, color="m", linestyle="--", label=r"$\epsilon_{th}$")
ax[1].legend()
ax[1].set_xlabel(r"$\epsilon$")
#ax[2].set_title("Variance f_1")

ax[2].plot(epsilons, stds_f2_push, "k.", label="F" )
ax[2].plot(epsilons, stds_f2_push, "k" )
ax[2].plot(epsilons, stds_f2_push_trunc, "b.", label="F trunc" )
ax[2].plot(epsilons, stds_f2_push_trunc, "b" )
ax[2].plot(0, std_f2_poisson, "r.", label=" Poisson")
ax[2].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[2].axvline(epsilon_th, color="m", linestyle="--", label=r"$\epsilon_{th}$")
ax[2].legend()
ax[2].set_xlabel(r"$\epsilon$")
#ax[3].set_title("Variance f_2")

ax[3].plot(epsilons, stds_f4_push, "k.", label="F" )
ax[3].plot(epsilons, stds_f4_push, "k" )
ax[3].plot(epsilons, stds_f4_push_trunc, "b.", label="F trunc" )
ax[3].plot(epsilons, stds_f4_push_trunc, "b" )
ax[3].plot(0, std_f4_poisson, "r.", label=" Poisson")
ax[3].axvline(epsilon_special, color="g", linestyle="--", label=r"$\epsilon_0$")
ax[3].axvline(epsilon_th, color="m", linestyle="--", label=r"$\epsilon_{th}$")
ax[3].legend()
ax[3].set_xlabel(r"$\epsilon$")
#ax[4].set_title("Variance f_4")
plt.show()


